# Introduction 

Example notebook how to set voltages with the Stahl bias sources

In [1]:
from qcodes.instrument_drivers.stahl import Stahl

In [2]:
stahl = Stahl("stahl", "ASRL3")

Connected to: Stahl HV (serial:171, firmware:None) in 0.19s


In [3]:
stahl.channel[4].voltage(2)
v = stahl.channel[4].voltage()
print(v)

2.002


In [4]:
stahl.channel[4].voltage(-2)
v = stahl.channel[4].voltage()
print(v)

-1.997


In [5]:
stahl.channel[4].voltage(0)
v = stahl.channel[4].voltage()
print(v)

0.001


In [6]:
stahl.channel[0].current()

1e-06

In [7]:
stahl.channel[0].current.unit

'A'

In [8]:
stahl.temperature()

25.9

In [9]:
stahl.channel[1].is_locked()

False

In [10]:
stahl.output_type

'bipolar'